In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import syft as sy
import copy
import numpy as np
import time
from opacus import PrivacyEngine
import time
from datetime import timedelta
from datetime import datetime

import importlib
importlib.import_module('FLDataset')
from FLDataset import load_dataset, getActualImgs, CovidDataset, Rescale, ToTensor
from utils import averageModels, averageGradients
from torch.utils.tensorboard import SummaryWriter

In [2]:
class Arguments():
    def __init__(self):
        self.images = 3012
        self.clients = 10
        self.rounds = 1000
        self.epochs = 1
        self.local_batches = 16
        self.lr = 0.02
        self.dropout = 0.5
        self.C = 0.9
        self.drop_rate = 0.1
        self.torch_seed = 0
        self.log_interval = 10
        self.iid = 'iid'
        self.split_size = int(self.images / self.clients)
        self.samples = self.split_size / self.images 
        self.use_cuda = True
        self.save_model = True
        self.save_model_interval = 200

args = Arguments()

use_cuda = args.use_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


cuda


In [3]:
hook = sy.TorchHook(torch)
clients = []

for i in range(args.clients):
    clients.append({'hook': sy.VirtualWorker(hook, id="client{}".format(i+1))})

print(clients)
print("number of clients : ", len(clients))

[{'hook': <VirtualWorker id:client1 #objects:0>}, {'hook': <VirtualWorker id:client2 #objects:0>}, {'hook': <VirtualWorker id:client3 #objects:0>}, {'hook': <VirtualWorker id:client4 #objects:0>}, {'hook': <VirtualWorker id:client5 #objects:0>}, {'hook': <VirtualWorker id:client6 #objects:0>}, {'hook': <VirtualWorker id:client7 #objects:0>}, {'hook': <VirtualWorker id:client8 #objects:0>}, {'hook': <VirtualWorker id:client9 #objects:0>}, {'hook': <VirtualWorker id:client10 #objects:0>}]
number of clients :  10


In [4]:
global_train, global_test, train_group, test_group = load_dataset(args.clients, args.iid)

In [5]:
print(len(global_train))
print(type(global_train))
print(len(global_test))
print(type(global_test))
print(len(train_group))
print(type(train_group))
print(len(test_group))
print(type(test_group))

3012
<class 'FLDataset.CovidDataset'>
753
<class 'FLDataset.CovidDataset'>
10
<class 'dict'>
10
<class 'dict'>


In [6]:
for inx, client in enumerate(clients):
    trainset_ind_list = list(train_group[inx])
    print("len(client", str(inx), "train set) = ", len(trainset_ind_list))
    client['trainset'] = getActualImgs(global_train, trainset_ind_list, args.local_batches)
    client['testset'] = getActualImgs(global_test, list(test_group[inx]), args.local_batches)
    client['samples'] = len(trainset_ind_list) / args.images

len(client 0 train set) =  301
len(client 1 train set) =  301
len(client 2 train set) =  301
len(client 3 train set) =  301
len(client 4 train set) =  301
len(client 5 train set) =  301
len(client 6 train set) =  301
len(client 7 train set) =  301
len(client 8 train set) =  301
len(client 9 train set) =  301


In [7]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
global_test_dataset = CovidDataset('./test.csv', transform=transforms.Compose([Rescale(64), ToTensor()]))
global_test_loader = DataLoader(global_test_dataset, batch_size=args.local_batches, shuffle=True)

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(30*30*64, 128)
        self.fc2 = nn.Linear(128, 3)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 30*30*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=args.dropout)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [9]:
def ClientUpdate(args, device, client):
    client['model'].train()
#     client['model'].send(client['hook'])
    
    for epoch in range(1, args.epochs + 1):
        for batch_idx, (data, target) in enumerate(client['trainset']):
            data, target = data.to(device), target.to(device)
            client['optim'].zero_grad()
            output = client['model'](data.float())
            loss = F.nll_loss(output, target.squeeze(1))
            loss.backward()
            client['optim'].step()
            
            if batch_idx % args.log_interval == 0 or batch_idx==len(client['trainset'])-1:
#                 loss = loss.get() 
                print('Model [{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    client['hook'].id,
                    epoch, (batch_idx+1) * args.local_batches, len(client['trainset']) * args.local_batches, 
                    100. * (batch_idx+1) / len(client['trainset']), loss))
                
#     client['model'].get() 

In [10]:
def test(args, model, device, test_loader, name):
    model.eval()   
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for d in test_loader:
            data = d['image']
            target = d['label']
            data, target = data.to(device), target.to(device)
            if(str(device)=='cuda'):
                model.cuda()
            output = model(data.float())
            test_loss += F.nll_loss(output, target.squeeze(1), reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss for {} model: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        name, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return 100. * correct / len(test_loader.dataset)

In [11]:
writer = SummaryWriter()

In [ ]:
torch.manual_seed(args.torch_seed)
global_model = Net()

for client in clients:
    torch.manual_seed(args.torch_seed)
    client['model'] = Net().to(device)
    client['optim'] = optim.SGD(client['model'].parameters(), lr=args.lr)
    client['pengine'] = PrivacyEngine(
                                       client['model'],
                                       batch_size=args.local_batches,
                                       sample_size=len(client['trainset']),
                                       alphas=range(2,32),
                                       noise_multiplier=0,
                                       max_grad_norm=1000
                                    )
    client['pengine'].attach(client['optim']) 
    
# start training model
training_start_time = time.time()
for fed_round in range(args.rounds):
    print("")
    print("===================================================================")
    print("[round] = ", fed_round+1, "/", args.rounds)
    print("===================================================================")
    
    round_train_start_time = time.time()
    
#     uncomment if you want a randome fraction for C every round
#     args.C = float(format(np.random.random(), '.1f'))    
    
    # number of selected clients
    m = int(max(args.C * args.clients, 1))

    # Selected devices
    np.random.seed(fed_round)
    selected_clients_inds = np.random.choice(range(len(clients)), m, replace=False)
    selected_clients = [clients[i] for i in selected_clients_inds]
    
    # Active devices
    np.random.seed(fed_round)
    active_clients_inds = np.random.choice(selected_clients_inds, int((1-args.drop_rate) * m), replace=False)
    active_clients = [clients[i] for i in active_clients_inds]
    
    # Training 
    client_cnt = 0
    for client in active_clients:
        print("* [client count] = ", client_cnt+1 , "/", len(active_clients))
        client_train_start_time = time.time()
        ClientUpdate(args, device, client)
        client_cnt += 1
        client_train_time = round(time.time()-client_train_start_time)
        print("* [client_train_time] = ", str(timedelta(seconds=(client_train_time))))
        print("---------------------------------------------------------------")
    
#         # Testing 
#         for client in active_clients:
#             test(args, client['model'], device, client['testset'], client['hook'].id)
    
    # Averaging 
    global_model = averageModels(global_model, active_clients)
    
    # Testing the average model
    acc = test(args, global_model, device, global_test_loader, 'Global')
    writer.add_scalar("Accuracy/train", acc, fed_round)
    writer.flush()
            
    # Share the global model with the clients
    for client in clients:
        client['model'].load_state_dict(global_model.state_dict())
        
    # training time per round
    total_train_time = round(time.time()-training_start_time)
    round_train_time = round(time.time()-round_train_start_time)
    print("** [total train time]: ", str(timedelta(seconds=total_train_time)))
    print("** [round train time]: ", str(timedelta(seconds=round_train_time)))
    
    if (args.save_model and fed_round%args.save_model_interval==0 and fed_round!=0):
        now = datetime.now() 
        date = now.strftime("%Y_%m_%d_%H%M")
        torch.save(global_model.state_dict(), date + "_FedAvg_with_DP_round_" + str(fed_round) + ".pth")
        print("model saved : "+ date +"_FedAvg_with_DP_round_" + str(fed_round) + ".pth")


/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/opacus/privacy_engine.py:518: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  "The sample rate will be defined from ``batch_size`` and ``sample_size``."
/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/opacus/privacy_engine.py:195: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_rng`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/opacus/privacy_engine.py:518: UserWarning: The sample rate will be defined from ``batch_size`` and ``sample_size``.The returned privacy budget will be incorrect.
  "The sample rate will be defined from ``batch_size`` 


[round] =  1 / 1000
* [client count] =  1 / 8


/home/citi302/anaconda3/envs/FLcourse/lib/python3.7/site-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Model [client4] Train Epoch: 1 [16/304 (5%)]	Loss: 12.480604
Model [client4] Train Epoch: 1 [176/304 (58%)]	Loss: 1.027103
Model [client4] Train Epoch: 1 [304/304 (100%)]	Loss: 2.849122
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  2 / 8
Model [client5] Train Epoch: 1 [16/304 (5%)]	Loss: 15.030823
Model [client5] Train Epoch: 1 [176/304 (58%)]	Loss: 1.035191
Model [client5] Train Epoch: 1 [304/304 (100%)]	Loss: 0.984815
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  3 / 8
Model [client9] Train Epoch: 1 [16/304 (5%)]	Loss: 13.402448
Model [client9] Train Epoch: 1 [176/304 (58%)]	Loss: 0.915605
Model [client9] Train Epoch: 1 [304/304 (100%)]	Loss: 0.539440
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  4 / 8
Model [client2] Train Epoch: 1 [16/304 (5%)]	Loss: 10.331553
Model [clien

Model [client8] Train Epoch: 1 [16/304 (5%)]	Loss: 1.076552
Model [client8] Train Epoch: 1 [176/304 (58%)]	Loss: 1.059110
Model [client8] Train Epoch: 1 [304/304 (100%)]	Loss: 1.047856
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  2 / 8
Model [client2] Train Epoch: 1 [16/304 (5%)]	Loss: 1.074989
Model [client2] Train Epoch: 1 [176/304 (58%)]	Loss: 1.018682
Model [client2] Train Epoch: 1 [304/304 (100%)]	Loss: 0.990888
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  3 / 8
Model [client1] Train Epoch: 1 [16/304 (5%)]	Loss: 0.987839
Model [client1] Train Epoch: 1 [176/304 (58%)]	Loss: 1.047643
Model [client1] Train Epoch: 1 [304/304 (100%)]	Loss: 0.975311
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  4 / 8
Model [client10] Train Epoch: 1 [16/304 (5%)]	Loss: 1.097543
Model [client10

...

Model [client1] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098790
Model [client1] Train Epoch: 1 [176/304 (58%)]	Loss: 1.098358
Model [client1] Train Epoch: 1 [304/304 (100%)]	Loss: 1.098522
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  2 / 8
Model [client10] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098270
Model [client10] Train Epoch: 1 [176/304 (58%)]	Loss: 1.097824
Model [client10] Train Epoch: 1 [304/304 (100%)]	Loss: 1.097267
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  3 / 8
Model [client2] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098643
Model [client2] Train Epoch: 1 [176/304 (58%)]	Loss: 1.098600
Model [client2] Train Epoch: 1 [304/304 (100%)]	Loss: 1.097984
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  4 / 8
Model [client9] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098426
Model [client

Model [client6] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098709
Model [client6] Train Epoch: 1 [176/304 (58%)]	Loss: 1.099794
Model [client6] Train Epoch: 1 [304/304 (100%)]	Loss: 1.097256
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  2 / 8
Model [client3] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098593
Model [client3] Train Epoch: 1 [176/304 (58%)]	Loss: 1.098467
Model [client3] Train Epoch: 1 [304/304 (100%)]	Loss: 1.097570
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  3 / 8
Model [client4] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098538
Model [client4] Train Epoch: 1 [176/304 (58%)]	Loss: 1.097210
Model [client4] Train Epoch: 1 [304/304 (100%)]	Loss: 1.098181
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  4 / 8
Model [client2] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098593
Model [client2] 

Model [client5] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098572
Model [client5] Train Epoch: 1 [176/304 (58%)]	Loss: 1.096330
Model [client5] Train Epoch: 1 [304/304 (100%)]	Loss: 1.097117
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  2 / 8
Model [client6] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098827
Model [client6] Train Epoch: 1 [176/304 (58%)]	Loss: 1.105113
Model [client6] Train Epoch: 1 [304/304 (100%)]	Loss: 1.095492
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  3 / 8
Model [client1] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098749
Model [client1] Train Epoch: 1 [176/304 (58%)]	Loss: 1.099233
Model [client1] Train Epoch: 1 [304/304 (100%)]	Loss: 1.099136
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  4 / 8
Model [client7] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098775
Model [client7] 

Model [client8] Train Epoch: 1 [176/304 (58%)]	Loss: 1.098944
Model [client8] Train Epoch: 1 [304/304 (100%)]	Loss: 1.098447
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  2 / 8
Model [client7] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098544
Model [client7] Train Epoch: 1 [176/304 (58%)]	Loss: 1.099567
Model [client7] Train Epoch: 1 [304/304 (100%)]	Loss: 1.097368
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  3 / 8
Model [client5] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098540
Model [client5] Train Epoch: 1 [176/304 (58%)]	Loss: 1.098282
Model [client5] Train Epoch: 1 [304/304 (100%)]	Loss: 1.101011
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  4 / 8
Model [client6] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098397
Model [client6] Train Epoch: 1 [176/304 (58%)]	Loss: 1.096592
Model [client6

Model [client1] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098437
Model [client1] Train Epoch: 1 [176/304 (58%)]	Loss: 1.097984
Model [client1] Train Epoch: 1 [304/304 (100%)]	Loss: 1.103695
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  2 / 8
Model [client7] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098582
Model [client7] Train Epoch: 1 [176/304 (58%)]	Loss: 1.103007
Model [client7] Train Epoch: 1 [304/304 (100%)]	Loss: 1.098075
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  3 / 8
Model [client3] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098665
Model [client3] Train Epoch: 1 [176/304 (58%)]	Loss: 1.097127
Model [client3] Train Epoch: 1 [304/304 (100%)]	Loss: 1.101448
* [client_train_time] =  0:00:04
---------------------------------------------------------------
* [client count] =  4 / 8
Model [client9] Train Epoch: 1 [16/304 (5%)]	Loss: 1.098511
Model [client9] 

In [ ]:
# tensorboard open 
# tensorboard --logdir=/home/citi302/Desktop/Codefolder/FL_DP_covid/runs